## Experiments
### Running finetuning using entire VGG model VS using precomputed conv. model outputs

### Loading and Running the finetuning in plain old batches through the entire model

In [14]:
from Executor import *
# Reset the model, and run same finetuning using precomputed conv. model outputs. See if they match...
executor = ExecutorBuilder().\
    with_runID("first").\
    and_().\
    with_Vgg16().\
    and_().\
    train_batch_size(128). \
    and_(). \
    learn_rate(0.001).\
    and_().\
    data_on_path("../data/full/").\
    build()

Pretrained Vgg16 model loaded.
Found 6255 images belonging to 3 classes.
Found 1549 images belonging to 3 classes.
initialized training data from: ../data/full/train
initialized validation data from: ../data/full/valid
found number of softmax classes: 3


In [15]:
executor.tune_softmax_layer_for_epochs(3)

Epoch 1/3
6255/6255 [==============================] - 183s - loss: 1.8112 - acc: 0.4406 - val_loss: 1.0254 - val_acc: 0.5423
Epoch 2/3
6255/6255 [==============================] - 182s - loss: 1.4456 - acc: 0.4902 - val_loss: 0.9752 - val_acc: 0.5436
Epoch 3/3
6255/6255 [==============================] - 182s - loss: 1.2545 - acc: 0.5052 - val_loss: 0.9401 - val_acc: 0.5565
Vgg model finetuned.


<Executor.Executor instance at 0x7f3026add518>

### Performing same operation using precomputed conv. model outputs

In [3]:
executor.load_precomputed_conv_models()

loading precomputed conv. outputs...
done...


<Executor.Executor instance at 0x7f31007c5998>

In [8]:
# Get the linear model 
linearModel = executor.get_rescaled_fc_model(new_dropout=executor.dropout)

# Set all layers except the last as being trainable
i=0
for layer in linearModel.layers:
    i+=1
    if(i!=7):
        layer.trainable=False
    if layer.trainable:
        print("found trainable layer: ",layer)


getting rescaled fc model...
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
found dense layer. Distributing scaled weights..
done...
('updating dropout from: ', 0.5, ' to: ', 0.5)
('found trainable layer: ', <keras.layers.core.Dense object at 0x7f30e39c1690>)


In [9]:
# try fitting the model. Used an extremely small learning rate... still no avail
linearModel.compile(optimizer=Adam(lr=0.000001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 7s - loss: 1.9735 - acc: 0.3845 - val_loss: 1.2826 - val_acc: 0.4835
Epoch 2/5
6255/6255 [==============================] - 7s - loss: 1.8062 - acc: 0.4045 - val_loss: 1.2044 - val_acc: 0.4816
Epoch 3/5
6255/6255 [==============================] - 7s - loss: 1.7036 - acc: 0.4179 - val_loss: 1.1815 - val_acc: 0.4835
Epoch 4/5
6255/6255 [==============================] - 7s - loss: 1.6493 - acc: 0.4139 - val_loss: 1.1535 - val_acc: 0.4835
Epoch 5/5
6255/6255 [==============================] - 7s - loss: 1.5510 - acc: 0.4203 - val_loss: 1.1450 - val_acc: 0.4822


In [10]:
linearModel.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 1.4943 - acc: 0.4261 - val_loss: 1.1434 - val_acc: 0.4822
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 1.5139 - acc: 0.4257 - val_loss: 1.1411 - val_acc: 0.4842
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 1.4811 - acc: 0.4349 - val_loss: 1.1393 - val_acc: 0.4829
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 1.5133 - acc: 0.4211 - val_loss: 1.1360 - val_acc: 0.4822
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 1.5068 - acc: 0.4264 - val_loss: 1.1356 - val_acc: 0.4803


In [11]:
linearModel.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics = ['accuracy'])
linearModel.fit(executor.train_precomputed, executor.train_labels, batch_size=executor.batch_size,
                nb_epoch= 5, validation_data=(executor.val_precomputed, executor.val_labels))

Train on 6255 samples, validate on 1549 samples
Epoch 1/5
6255/6255 [==============================] - 3s - loss: 1.4818 - acc: 0.4257 - val_loss: 1.1028 - val_acc: 0.4771
Epoch 2/5
6255/6255 [==============================] - 3s - loss: 1.3043 - acc: 0.4371 - val_loss: 1.1057 - val_acc: 0.4868
Epoch 3/5
6255/6255 [==============================] - 3s - loss: 1.2354 - acc: 0.4531 - val_loss: 1.0777 - val_acc: 0.4693
Epoch 4/5
6255/6255 [==============================] - 3s - loss: 1.1974 - acc: 0.4540 - val_loss: 1.0890 - val_acc: 0.4881
Epoch 5/5
6255/6255 [==============================] - 3s - loss: 1.1599 - acc: 0.4638 - val_loss: 1.0998 - val_acc: 0.4099
